# Modelo usado para la demo
### Adrián Arnaiz
Hemos elegido para la demo el modelo **MLP con 10 capas ocultas, que funcionará (entrenado) con las características tipo embeddings extraídas de los audios tipo read_text mediante VGGish.** 

Hemos elegido este modelo por varias razones:
* Es el que **mejor funciona sin dividir por sexos** (posible mejora futura: derivar a diferentes modelos según el sexo del paciente)
* Aporta novedad al estado del arte: Saca las **caracteristicas mediante VGGish** (red neuronal con capas convolucionales) y **predice mediante MLP**, el cual es otra red neuronal, un perceptrón multicapa (Multi Layer Perceptron).
* La extracción de características mediante **VGGish** es considerablemente más rapida que mediante la librería Disvoice. En contra, las características que saca VGGish no sabemos que son, mediante que las de Disvoice sí que son interpretables.
* Los audios tipo **read_text**, como ya hemos visto en el estado del arte, nos dan más información acerca de la enfermedad. Sacar conclusiones de una vocal sostenida puede ser ventajista, tal y como hemos visto y está explicado en la documentación.

-----

### Cross Validation 10 Fold
Haremos validación cruzada para ver los resultados del modelo con esta técnica. Sin embargo, para guardar luego el modelo, deberemos hacer un fit normal ya que con cross_val_score no se nos permite guardar un modelo entrenado.

In [168]:
import numpy as np
import cargaDatosVggishEmbeddings

import numpy as np
import pandas as pd
import importlib
from sklearn.base import clone
from sklearn.metrics import SCORERS, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
import warnings
import pickle
warnings.filterwarnings('ignore')

In [39]:
loader = cargaDatosVggishEmbeddings
read_data = loader.load_vggish_embed_rt()

In [46]:
X=read_data.data
y=read_data.target
X.shape, y.shape

((100, 258), (100,))

In [47]:
pipe = Pipeline([('norm', MinMaxScaler()), ('clf', MLPClassifier(hidden_layer_sizes=10))])

In [48]:
res = cross_val_score(pipe,X,y,cv=10,scoring='roc_auc')
res

array([0.72, 0.88, 0.88, 0.96, 0.88, 1.  , 1.  , 1.  , 0.76, 0.84])

In [49]:
res.mean()

0.892

---
### Guardamos el modelo
Hacemos la validación cruzada a mano, y guardamos uno de los modelos realizados en un sprint concreto.

In [123]:
fold = StratifiedKFold(n_splits=10)

In [125]:
modelos = dict()
i=0
aucs = []
for train_index, test_index in fold.split(X, y):
    pipe = Pipeline([('norm', MinMaxScaler()), ('clf', MLPClassifier(hidden_layer_sizes=10))])
    #Dividimos por splits
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #Entrenamos
    mod = pipe.fit(X_train, y_train)
    #Predecimos
    y_pred = pipe.predict(X_test)
    #Sacamos medida auc
    auc_score = roc_auc_score(y_pred, y_test)
    aucs.append(auc_score)
    #guardamos el modelo creado con este split
    modelos[i] = mod
    i+=1
aucs, np.mean(aucs)

([0.7777777777777778,
  0.9166666666666667,
  0.7083333333333334,
  0.9166666666666667,
  0.7777777777777778,
  0.9166666666666667,
  0.9166666666666667,
  1.0,
  0.7777777777777778,
  0.8],
 0.8508333333333334)

Elegimos el modelo con auc = 0.916. No elegimos el modelo con auc=1 ya que comprendemos que sobhreajusta. Un modelo con 100% de aciertop tampoco es fiable ya que entendemos que se sobreajusta a los datos. Por ello elegimos el modelo con auc 0.916, generaliza bien sin sobreajustar.

In [127]:
my_model = modelos[1]

In [169]:
filename = 'MLP10_vggish_rt.sav'
#guardamos
pickle.dump(my_model, open(filename, 'wb'))

---